<a href="https://colab.research.google.com/github/filispeen/blender-render-eevee-and-cycles/blob/main/blender_render_eevee_and_cycles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
**Make sure to read the instructions carefully!**

If you have other resources used in the Blender project and chose to *make all paths relative*, pack all of them into a zip archive. Alternatively, you can *pack all external file*.

* `blender_version` : Unable to change version beacause linux eevee render support only blender 3.4.
* `blend_file_path` : Path to the blend file after unpacking the zip archive. If blend file is used, this is automatically ignored.
___
* `upload_type` : Select the type of upload method. `gdrive_relative` pulls everything from the folder specified.
* `drive_path` : Path to your blend/zip file relative to the root of your Google Drive if `google_drive` is selected. Must  state the file and its extension (.zip/.blend) **unless** `gdrive_relative` is selected.
* `url_blend` : Specify the URL to the blend/zip file if `url` is selected.
___
* `animation` : Specify whether animation or still image is rendered. If **still image** is used, put the frame number in `start_frame`.
* `start_frame, end_frame` : Specify the start and end frame for animation. You may put same value such as zero for both input to set the default frame in the blend file.
___
* `download_type` : Select the type of download method. `gdrive_direct` enables the frames to be outputted directly to Google Drive (zipping will be disabled).
* `output_name` : Name of the output frames, **do NOT include .blend!** (## for frame number)
* `zip_files` : Archive multiple animation frames automatically into a zip file.
* `drive_output_path` : Path to your frames/zip file in Google Drive.
___
* `gpu_enabled, cpu_enabled` : Toggle GPU and CPU for rendering. CPU might give a slight boost in rendering time but may varies depend on the project.
* `optix_enable` : Enable OptiX which may boost performance, may be incompatible depending on the version of blender, project and GPU allocated

After you are done, go to Runtime > Run All (Ctrl + F9) and upload your files or have Google Drive authorised below. See the [GitHub repo](https://github.com/filispeen/blender-render-eevee-and-cycles) for more information.

In [ ]:
#@title Settings
blend_file_path = 'path/to/blend/file.blend' #@param {type:"string"}
upload_type = "google_drive" #@param ["direct", "google_drive"]
#@markdown
output_folder = 'path/there/frames/saves/' #@param {type:"string"}
download_type = "google_drive" #@param ["direct", "google_drive"]
#@markdown
engine = "EEVEE" #@param ["EEVEE", "CYCLES"]
frame_start = "1" #@param {type:"string"}
frame_end = "250" #@param {type:"string"}
animation = False #@param {type:"boolean"}
#@markdown
optix_enabled = False #@param{type:"boolean"}
cpu_enabled = False #@param{type:"boolean"}

In [ ]:
#@title Checking for GPU
import os
import shutil

!apt install toilet

!toilet SCRIPT BY 
!toilet FILISPEEN

gpu = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print("Current GPU: " + gpu[0])

if gpu[0] == "Tesla K80" and optix_enabled:
  print("OptiX disabled because of unsupported GPU")
  optix_enabled = False

if gpu[0] == "NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running." and engine == "EEVEE":
  raise SystemExit("EEVEE engine workig only with GPU. Swtith to CYCLES engine.")

def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(os.path.relpath(root,start = os.curdir), name)

In [ ]:
#@title Connect Google Drive to Google Colab
from google.colab import drive

if upload_type == 'google_drive' or upload_type == 'gdrive_relative' or download_type == 'google_drive' or download_type == 'gdrive_direct':
    drive.mount('/drive')

if upload_type == 'direct':
    uploaded = files.upload()
    for fn in uploaded.keys():
        uploaded_filename = fn
        !cp /content/*.blend /content/blendfile.blend
elif upload_type == 'url':
    !wget -nc $url_blend
    uploaded_filename = os.path.basename(url_blend)
elif upload_type == 'google_drive':
    shutil.copy('/drive/My Drive/' + blend_file_path, '.')
    uploaded_filename = os.path.basename(blend_file_path)

In [ ]:
#@title Download Blender 3.4 from Blender Repository. 
!wget -nc https://builder.blender.org/download/daily/blender-3.4.1-stable+v34.ef9ca44dee7f-linux.x86_64-release.tar.xz -O BlenderDownload.tar.xz
#Copy Blender archive
!cp /content/drive/MyDrive/Blender/BlenderDownload.tar.xz /content/BlenderDownload.tar.xz
#unarchive blender
!tar xf /content/BlenderDownload.tar.xz

In [ ]:
#@title Enable GPU rendering for cycles (or add custom properties here)
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found:',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

renderer = "CUDA"
if optix_enabled:
    print("Note: You're currently using OptiX renderer. If an error occurred, the current GPU (example: Tesla K80) is not supported and you need to switch back to Cycles.")
    renderer = "OPTIX"
if engine == "EEVEE"
    print("Note: You`re currently using cpu")
    cpu_enabled = False
    renderer = "CUDA"


In [ ]:
#@title Rendering
shutil.copy('/content/drive/MyDrive/' + blend_file_path, 'blendfile.blend')

!mkdir /content/render

output('/content/drive/MyDrive/' + output_folder)
print(output)

if engine == "EEVEE"
if animation:
    if frame_start == frame_end:
        !./Blender/blender -b '/content/blendfile.blend' -noaudio -o '/content/render/' -a -F 'PNG'
    else:
        !./Blender/blender -b '/content/blendfile.blend' -noaudio -o '/content/render/' -a -s $frame_start -e $frame_end -F 'PNG'
else:
    !./Blender/blender -b '/content/blendfile.blend' -noaudio -o '/content/render/' -s $frame_start -F 'PNG'

if engine == "CYCLES"
if animation:
    if frame_start == frame_end:
        !./$blender_version/blender -b '/content/blendfile.blend' -P setgpu.py -E CYCLES -o '/content/render/' -noaudio -a -- --cycles-device "{renderer}"
    else:
        !./$blender_version/blender -b '/content/blendfile.blend' -P setgpu.py -E CYCLES -o '/content/render/' -noaudio -s $frame_start -e $frame_end -a -- --cycles-device "{renderer}"
else:
    !./$blender_version/blender -b '/content/blendfile.blend' -P setgpu.py -E CYCLES -o '/content/render/' -noaudio -f $frame_end -- --cycles-device "{renderer}"

In [ ]:
#@title Saving frames
%cd /content/render/


if download_type == "google_drive"
    shutil.make_archive('.', 'zip', '/content/render/output')
    shutil.copy('output.zip', '/content/drive/MyDrive/' + output_folder)

if download_type == "direct"
    shutil.make_archive('.', 'zip', '/content/render/output')
    files.download('output.zip')

else:
    raise SystemExit("No frames are rendered.")

## Disclaimer
Google Colab is targeted to researchers and students to run AI/ML tasks, data analysis and education, not rendering 3D scenes. Because the computing power provided are free, the usage limits, idle timeouts and speed of the rendering may varies time by time. [Colab Pro and Colab Pro+](https://colab.research.google.com/signup) are available for those who wanted to have more powerful GPU and longer runtimes for rendering. See the [FAQ](https://research.google.com/colaboratory/faq.html) for more info. In some cases, it might be faster to use an online Blender renderfarm.

## License
```
MIT License

Copyright (c) 2020-2023 ynshung

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```